<a href="https://colab.research.google.com/github/tms1337/nootebooks/blob/master/Tested_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install matplotlib
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def fetch_ticker_data(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date)
    return data

## Loading Tickers

In [ ]:
df=pd.read_csv('tickers.csv')

In [ ]:
tickers = df['Ticker'].tolist()

## Fetching Tickers data

In [ ]:
data = fetch_ticker_data(tickers, '2022-07-06', '2023-07-06')

## Saving the original data

In [ ]:
data.to_csv('original_data.csv')

In [ ]:
data.head(10)

## Synthetic data

In [ ]:
# def generate_synthetic_data(df):
#     # Define the window size for calculating rolling standard deviation
#     window_size = 11

#     # Copy the dataframe
#     synthetic_df = df.copy()

#     # Loop through each column in the dataframe
#     for column in synthetic_df.columns:

#         # Calculate the rolling standard deviation for the current column
#         std_dev = synthetic_df[column].rolling(window_size).std()

#         # Create a series of random numbers between -1 and 1
#         random_series = np.random.uniform(low=-1, high=1, size=len(synthetic_df[column]))

#         # Generate synthetic data by multiplying original data with the standard deviation and random number
#         synthetic_df[column] = synthetic_df[column] + (0.01 + std_dev * random_series)

#     return synthetic_df

## Change random range here


In [ ]:
def generate_synthetic_data(df):
    # Define the window size for calculating rolling standard deviation
    window_size = 11

    # Copy the dataframe
    synthetic_df = df.copy()

    # Generate a series of random numbers between -1 and 1 for each day
    random_series = np.random.uniform(low=-1.0, high=1.0, size=len(synthetic_df))

    # Loop through each ticker in the dataframe
    tickers = df.columns.get_level_values(1).unique()
    for ticker in tickers:
        # Loop through each attribute of the ticker
        attributes = df.columns.get_level_values(0).unique()
        for attr in attributes:
            # Calculate the rolling standard deviation for the current attribute of the ticker
            std_dev = synthetic_df[(attr, ticker)].rolling(window_size).std()

            # Generate synthetic data by adding to original data the product of the standard deviation and the same random number for each day
            synthetic_df[(attr, ticker)] += 0.01 + (std_dev * random_series)

    return synthetic_df


In [ ]:
synthetic_data = generate_synthetic_data(data)

In [ ]:
data.tail(5)

In [ ]:
synthetic_data.tail(5)

## Data Cleaning

In [ ]:
synthetic_data.fillna(method='bfill',inplace=True)

## Saving Synthetic data

In [ ]:
synthetic_data.to_csv('synthetic_data.csv')

### Load synthetic data file:

In [ ]:
# Load your data (adjust path if necessary)
synthetic_data = pd.read_csv('synthetic_data.csv', header=[0, 1], index_col=0, parse_dates=True)

synthetic_data.sample(10)

In [ ]:
def compare_data(df_original, df_synthetic, ticker):
    # Define the attributes that we are interested in
    attributes = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

    # Create a new DataFrame to hold the data for the specific ticker
    df_ticker_original = pd.DataFrame(index=df_original.index)
    df_ticker_synthetic = pd.DataFrame(index=df_synthetic.index)

    for attr in attributes:
        # Find the start column for the specific attribute of the ticker
        start_col_original = df_original.columns.get_loc((attr, ticker))
        start_col_synthetic = df_synthetic.columns.get_loc((attr, ticker))

        # Select the data for the specific attribute of the ticker
        df_ticker_original[attr] = df_original.iloc[:, start_col_original]
        df_ticker_synthetic[attr] = df_synthetic.iloc[:, start_col_synthetic]

    # Ensure data types are float
    for col in df_ticker_original.columns:
        df_ticker_original[col] = df_ticker_original[col].astype(float)
    for col in df_ticker_synthetic.columns:
        df_ticker_synthetic[col] = df_ticker_synthetic[col].astype(float)

    # Plot the data
    fig, ax = plt.subplots(2, 1, figsize=(12, 8))

    # Plot the closing prices
    ax[0].plot(df_ticker_original.index, df_ticker_original['Close'], label='Original')
    ax[0].plot(df_ticker_synthetic.index, df_ticker_synthetic['Close'], label='Synthetic')
    ax[0].set_ylabel('Close Price')
    ax[0].legend()

    # Plot the volume
    ax[1].bar(df_ticker_original.index, df_ticker_original['Volume'], label='Original')
    ax[1].bar(df_ticker_synthetic.index, df_ticker_synthetic['Volume'], label='Synthetic')
    ax[1].set_ylabel('Volume')
    ax[1].set_yscale('log')  # Set y-axis to logarithmic scale
    ax[1].legend()

    plt.title(f'{ticker} Price and Volume')
    plt.show()


In [ ]:
compare_data(data, synthetic_data, 'WYNN')

In [ ]:
#!pip install mpl-finance
import mplfinance as mpf
from mplfinance import _styles
_styles.available_styles()

## ATR Calculation

In [ ]:
def calculate_atr(df, n=10):
    data = df.copy()
    high = data[('High')]
    low = data[('Low')]
    close = data[('Close')]
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.shift())
    data['tr2'] = abs(low - close.shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    atr = tr.rolling(n).mean()
    return atr

## Data transformation
#### Extracting out from MultiIndexer data

In [ ]:
def plot_data(df, ticker):
    # Define the attributes that we are interested in
    attributes = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

    # Create a new DataFrame to hold the data for the specific ticker
    df_ticker = pd.DataFrame(index=df.index)

    for attr in attributes:
        # Find the start column for the specific attribute of the ticker
        start_col = df.columns.get_loc((attr, ticker))

        # Select the data for the specific attribute of the ticker
        df_ticker[attr] = df.iloc[:, start_col]

    # Ensure data types are float
    for col in df_ticker.columns:
        df_ticker[col] = df_ticker[col].astype(float)

    # Add ATR calculation to the DataFrame
    df_ticker['ATR'] = calculate_atr(df_ticker)

    # Plot
    mpf.plot(df_ticker,
             type='ohlc', #candle , renko, ohlc
             ema=(18,50),
             volume=True,
             figscale=1.5,
             style='yahoo',
             title=f'{ticker} Price')

In [ ]:
# Test the function with your data
plot_data(synthetic_data, 'AAPL')

In [ ]:
plot_data(data, 'AAPL')

In [ ]:
import matplotlib.pyplot as plt
from mplfinance.original_flavor import candlestick_ohlc
import mplfinance as mpf

def plot_data_with_ATR(df_original, df_synthetic, ticker):
    # Define the attributes that we are interested in
    attributes = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

    # Create new DataFrames to hold the data for the specific ticker
    df_ticker_original = df_original.xs(ticker, level=1, axis=1)[attributes]
    df_ticker_synthetic = df_synthetic.xs(ticker, level=1, axis=1)[attributes]

    # Add ATR calculation to the DataFrames
    df_ticker_original['ATR'] = calculate_atr(df_ticker_original)
    df_ticker_synthetic['ATR'] = calculate_atr(df_ticker_synthetic)

    # Create subplots
    fig, ax = plt.subplots(3, 1, sharex=True, figsize=(15, 10))

    # Plot the closing prices
    ax[0].plot(df_ticker_original.index, df_ticker_original['Close'], label='Original')
    ax[0].plot(df_ticker_synthetic.index, df_ticker_synthetic['Close'], label='Synthetic')
    ax[0].set_ylabel('Close Price')
    ax[0].legend()

    # Plot the volume
    ax[1].bar(df_ticker_original.index, df_ticker_original['Volume'], label='Original')
    ax[1].bar(df_ticker_synthetic.index, df_ticker_synthetic['Volume'], label='Synthetic')
    ax[1].set_ylabel('Volume')
    ax[1].legend()

    # Plot the ATR
    ax[2].plot(df_ticker_original.index, df_ticker_original['ATR'], label='Original')
    ax[2].plot(df_ticker_synthetic.index, df_ticker_synthetic['ATR'], label='Synthetic')
    ax[2].set_ylabel('ATR')
    ax[2].legend()

    plt.show()

In [ ]:
plot_data_with_ATR(data, synthetic_data, 'AAPL')

In [ ]:
# type='candle', style='yahoo', mav=(5,20), volume=True, addplot=plots, panel_ratios=(3,1,3,3), figscale=1.5

In [ ]:
# df_ticker = pd.DataFrame(index=df.index)

# for attr in attributes:
#     # Find the start column for the specific attribute of the ticker
#     start_col = df.columns.get_loc((attr, 'ACGL'))
#     print(start_col)

#     # Select the data for the specific attribute of the ticker
#     df_ticker[attr] = df.iloc[:, start_col]

In [ ]:
# start_col

In [ ]:
# df_ticker